# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:
import sys


import math
import os
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from os.path import exists

In [2]:
# any general notebook setup, like log formatting

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "taxi_csv/uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [4]:
def calculate_distance(from_coord, to_coord):
    x1 = float(from_coord[0])
    y1 = float(from_coord[1])
    x2 = float(to_coord[0])
    y2 = float(to_coord[1])
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    


In [5]:
def test_calculate_distance():
    p1 = (3,5)
    p2 = (6,2)
    dist = calculate_distance(p1,p2)
    print(dist)

In [6]:
def add_distance_column(dataframe):
    #print("add_distance_column")
    #print(type(dataframe))
    new_column_values = []
    #print(df.columns)
    for index, x in dataframe.iterrows():
        new_column_values.append(calculate_distance((x["pickup_latitude"],x["pickup_longitude"]),(x["dropoff_latitude"],x["dropoff_longitude"])))
    dataframe.insert(loc=len(dataframe.columns) ,column='distance',value=new_column_values)
    dataframe = dataframe[(dataframe["distance"] > 0)]
    
    return dataframe

### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [7]:
def find_taxi_csv_urls():
    years = [2009,2010,2011,2012,2013,2014,2015]
    content = requests.get(TAXI_URL)
    soup = bs4.BeautifulSoup(content.text, 'html.parser')
    
    ancors = soup.find_all("a")
    ancors_yellow = []
    
    for i in ancors:
        if 'title' in i.attrs.keys() and i["title"] == "Yellow Taxi Trip Records" and int(i["href"][-11:-7]) in years:
            ancors_yellow.append(i["href"])
    
    return ancors_yellow
    

def get_and_clean_month_taxi_data(url):
    name = url[url.rfind("/")+1:]
    if(exists("./taxi_csv/"+name[:-4]+"-short.csv")):
        df = pd.read_csv("./taxi_csv/"+name[:-4]+"-short.csv")
        return df
    else:
        df=pd.read_csv(url)
        p1 = NEW_YORK_BOX_COORDS[0]
        p2 = NEW_YORK_BOX_COORDS[1]
        x1 = p1[0]
        y1 = p1[1]
        x2 = p2[0]
        y2 = p2[1]
        df = df[(df["pickup_latitude"] > x1) & (df["pickup_latitude"] < x2) & (df["pickup_longitude"] > y1) & (df["pickup_longitude"] < y2)]
        df = df[(df["dropoff_latitude"] > x1) & (df["dropoff_latitude"] < x2) & (df["dropoff_longitude"] > y1) & (df["dropoff_longitude"] < y2)]
        df = df.dropna()
        df = df.sample(n=2381, random_state=1)
        df.to_csv("./taxi_csv/"+name[:-4]+"-short.csv")
        return df
'''
    if(exists("./taxi_csv/"+name)):
        return clean_taxi_file('./taxi_csv/'+name)
    else:
        r = requests.get(url)
        
        print(c.head)
        with open('./taxi_csv/'+name, 'wb') as f:
            f.write(r.content)
            
        
        
'''

            
'''
def clean_taxi_file(filename):
    data = pd.read_csv(filename)
    df = pd.DataFrame(data)
    p1 = NEW_YORK_BOX_COORDS[0]
    p2 = NEW_YORK_BOX_COORDS[1]
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    df = df[(df["pickup_latitude"] > x1) & (df["pickup_latitude"] < x2) & (df["pickup_longitude"] > y1) & (df["pickup_longitude"] < y2)]
    df = df[(df["dropoff_latitude"] > x1) & (df["dropoff_latitude"] < x2) & (df["dropoff_longitude"] > y1) & (df["dropoff_longitude"] < y2)]
    df = df.dropna()
    df = df.sample(n=2381, random_state=1)
    df.to_csv(filename[:-4]+"-short.csv")
    return df
'''

def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()[0:1] # done
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        #print(dataframe.head)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data


### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [8]:
def load_and_clean_uber_data(csv_file):
    
    data = pd.read_csv(csv_file)
    df = pd.DataFrame(data)
    p1 = NEW_YORK_BOX_COORDS[0]
    p2 = NEW_YORK_BOX_COORDS[1]
    x1 = p1[0]
    y1 = p1[1]
    x2 = p2[0]
    y2 = p2[1]
    df = df[(df["pickup_latitude"] > x1) & (df["pickup_latitude"] < x2) & (df["pickup_longitude"] > y1) & (df["pickup_longitude"] < y2)]
    df = df[(df["dropoff_latitude"] > x1) & (df["dropoff_latitude"] < x2) & (df["dropoff_longitude"] > y1) & (df["dropoff_longitude"] < y2)]
    df = df.drop(columns=["key","fare_amount","passenger_count"], axis=1)
    df = df.iloc[: , 1:]
    df = df.sample(n=2000, random_state=1)
    
    return df

def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    return uber_dataframe


### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [9]:
def clean_month_weather_data_hourly(csv_file):
    data = pd.read_csv(csv_file)
    df = pd.DataFrame(data)
    if "STATION" in df.columns:
        p1 = NEW_YORK_BOX_COORDS[0]
        p2 = NEW_YORK_BOX_COORDS[1]
        x1 = p1[0]
        y1 = p1[1]
        x2 = p2[0]
        y2 = p2[1]
        df = df[(df["LATITUDE"] > x1) & (df["LATITUDE"] < x2) & (df["LONGITUDE"] > y1) & (df["LONGITUDE"] < y2)]
        df = df[["DATE", "HourlyPrecipitation", "HourlyWindSpeed"]]
        df.columns = ["date","precipitation","wind"]
        df = df.dropna()
        #print(df.head)
        df = df.sample(n=300, random_state=1)
    
    return df
    
def clean_month_weather_data_daily(csv_file):
    data = pd.read_csv(csv_file)
    df = pd.DataFrame(data)
    if "STATION" in df.columns:
        p1 = NEW_YORK_BOX_COORDS[0]
        p2 = NEW_YORK_BOX_COORDS[1]
        x1 = p1[0]
        y1 = p1[1]
        x2 = p2[0]
        y2 = p2[1]
        df = df[(df["LATITUDE"] > x1) & (df["LATITUDE"] < x2) & (df["LONGITUDE"] > y1) & (df["LONGITUDE"] < y2)]
        df = df[["DATE", "DailyPrecipitation","DailyAverageWindSpeed"]]
        df.columns = ["date","precipitation","wind"]
        df = df.dropna()
        
    return df

    
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    
    weather_csv_files = []
    for f in os.listdir("./taxi_csv/weather"):
        weather_csv_files.append("./taxi_csv/weather/"+f)
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data



### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [10]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()
#print(hourly_weather.head)

C:\Users\rabee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rabee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rabee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rabee\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rabee\anaconda3\lib\site-packages\IPy

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [11]:
import sqlite3

if(exists("project.db")):
    os.remove("project.db")
if(exists("schema.sql")):
    os.remove("schema.sql")



#engine = db.create_engine(DATABASE_URL, echo=True)
#engine = db.create_engine(DATABASE_URL)#, echo=True)
connection = sqlite3.connect("project.db")

In [12]:
#Base.metadata.create_all(engine, checkfirst=True)

In [13]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
   id INTEGER PRIMARY KEY AUTOINCREMENT,
   date DATETIME,
   precipitation FLOAT(2),
   windspeed FLOAT(2)
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
   id INTEGER PRIMARY KEY AUTOINCREMENT,
   date DATETIME,
   precipitation FLOAT(2),
   windspeed FLOAT(2)
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
   id INTEGER PRIMARY KEY AUTOINCREMENT,
   date DATETIME,
   distance FLOAT(2)
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
   id INTEGER PRIMARY KEY AUTOINCREMENT,
   date DATETIME,
   distance FLOAT(2)
);
"""

In [14]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [15]:
# create the tables with the schema files
with connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [16]:
def dataframes_to_table(dic):
    y_taxi = dic["taxi_trips"]
    u_taxi = dic["uber_trips"]
    h_weather = dic["hourly_weather"]
    d_weather = dic["daily_weather"]
    
    with connection:
        
        for index, row in y_taxi.iterrows():
            date,time = row['tpep_pickup_datetime'].split()
            distance = row['distance']
            date = row['tpep_pickup_datetime']
            connection.execute("INSERT INTO taxi_trips (date,distance) VALUES(\""+date+"\","+str(distance)+")")
            
    with connection:
        for index, row in u_taxi.iterrows():            
            date,time,utc = row['pickup_datetime'].split()
            distance = row['distance']
            date = date + " " + time
            connection.execute("INSERT INTO uber_trips (date,distance) VALUES(\""+date+"\","+str(distance)+")")
            
    with connection:
        for index, row in h_weather.iterrows():
            date,time = row['date'].split("T")
            date = date + ' ' + time
            precipitation = str(row['precipitation'])
            if precipitation == "T":
                precipitation = "0"
            if precipitation[-1] == "s":
                precipitation = precipitation[:-1]
            windspeed = str(row['wind'])
            connection.execute("INSERT INTO hourly_weather (date,precipitation,windspeed) VALUES(\""+date+"\","+str(precipitation)+","+str(windspeed)+")")

    with connection:
        for index, row in d_weather.iterrows():
            date,time = row['date'].split("T")
            date = date + ' ' + time
            precipitation = str(row['precipitation'])
            if precipitation == "T":
                precipitation = "0"
            if precipitation[-1] == "s":
                precipitation = precipitation[:-1]
            windspeed = str(row['wind'])
            connection.execute("INSERT INTO daily_weather (date,precipitation,windspeed) VALUES(\""+date+"\","+str(precipitation)+","+str(windspeed)+")")

            
    
        

In [17]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather,
    "daily_weather": daily_weather,
}

dataframes_to_table(table_name_to_dataframe)

with connection:
    taxi_table = connection.execute('SELECT * FROM taxi_trips')
    uber_table = connection.execute('SELECT * FROM uber_trips')
    hour_table = connection.execute('SELECT * FROM hourly_weather')
    day_table = connection.execute('SELECT * FROM daily_weather')
        
    count = 0
    for r in taxi_table:
        print(r)
        count+=1
        if count == 10:
            break
            
    count = 0
    for r in uber_table:
        print(r)
        count+=1
        if count == 10:
            break
            
    count = 0
    for r in hour_table:
        print(r)
        count+=1
        if count == 10:
            break
            
    count = 0
    for r in day_table:
        print(r)
        count+=1
        if count == 10:
            break



(1, '2015-01-17 21:35:36', 0.0189796105953511)
(2, '2015-01-30 21:29:59', 0.05769467245394183)
(3, '2015-01-23 22:03:17', 0.025548057427168713)
(4, '2015-01-26 06:09:21', 0.01463714203847305)
(5, '2015-01-15 08:32:44', 0.024472644378540496)
(6, '2015-01-17 22:27:52', 0.0485657708434392)
(7, '2015-01-17 15:29:48', 0.01667579933108069)
(8, '2015-01-24 13:44:20', 0.006323512865039345)
(9, '2015-01-24 20:50:02', 0.0110676727604012)
(10, '2015-01-22 17:12:20', 0.02977101291148548)
(1, '2011-01-12 19:46:36', 0.02349602308902379)
(2, '2012-06-11 23:20:00', 0.06085557513490121)
(3, '2013-11-18 12:25:07', 0.013200148673404214)
(4, '2011-08-13 01:17:00', 0.0141109903975566)
(5, '2013-09-13 23:08:00', 0.04747195081309461)
(6, '2015-06-12 09:00:16', 0.020463353170840068)
(7, '2013-04-04 20:53:56', 0.015085611223943488)
(8, '2010-08-24 17:20:00', 0.010711540178706886)
(9, '2011-10-26 14:48:00', 0.017888504493111256)
(10, '2014-05-11 03:47:00', 0.036915334605013464)
(1, '2009-10-18 05:51:00', 0.03, 

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [18]:
def write_query_to_file(query, outfile):
    f = open(outfile,"w")
    f.write(query)
    f.close()

### Query N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [19]:
#hourly_weather (date,time,precipitation,windspeed)
#daily_weather (date,precipitation,windspeed)
#taxi_trips (date,distance)
#uber_trips (date,distance)

In [122]:
QUERY_1 = """
SELECT strftime('%H',date),COUNT(strftime('%H',date))
FROM taxi_trips
WHERE date >= "2009-01-01" AND date <= "2015-06-01"
GROUP BY strftime('%H',date)
ORDER BY COUNT(strftime('%H',date)) DESC
"""

QUERY_2 = """
SELECT substr('Sunday   Monday   Tuesday  WednesdayThursday Friday   Saturday ', 1 + 9*strftime('%w', date), 9) as dow ,count(strftime('%w',date))
FROM uber_trips
WHERE date >= "2009-01-01" AND date <= "2015-06-01"
GROUP BY strftime('%w',date)
ORDER BY COUNT(strftime('%w',date)) DESC
"""

QUERY_3 = """
SELECT distance
FROM
(SELECT distance
FROM taxi_trips
UNION ALL
SELECT distance
FROM uber_trips) AS T
ORDER BY distance ASC
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM (SELECT distance
        FROM taxi_trips
        UNION ALL
        SELECT distance
        FROM uber_trips)) * 19 / 20 - 1;
"""

QUERY_4 = """
SELECT DATE(date), count(strftime('%Y',date)), AVG(distance)
FROM 
(SELECT * from taxi_trips
UNION ALL
SELECT * from uber_trips)
WHERE strftime('%Y',date) = "2009"
GROUP BY DATE(date)
ORDER BY count(strftime('%Y',date)) DESC
LIMIT 10
"""

QUERY_5 = """
SELECT count(*)
FROM 
(SELECT *
FROM taxi_trips
UNION ALL
SELECT *
FROM uber_trips)
WHERE DATE(date) IN
        (SELECT DATE(date)
        FROM daily_weather
        WHERE strftime('%Y',date) = "2014"
        ORDER BY windspeed DESC
        LIMIT 10)
"""

QUERY_6 = """
SELECT *
FROM
    (SELECT DATE(date) as date, strftime('%H',date) as HR, precipitation, windspeed
    FROM hourly_weather
    WHERE DATE(date) >= "2012-10-23" AND DATE(date) <= "2012-10-30") AS W
    LEFT JOIN
    (SELECT DATE(date) as date, strftime('%H',date) AS HR, distance
    FROM 
    (SELECT *
    FROM taxi_trips
    WHERE DATE(date) >= "2012-10-23" AND DATE(date) <= "2012-10-30"
    UNION ALL
    SELECT *
    FROM uber_trips
    WHERE DATE(date) >= "2012-10-23" AND DATE(date) <= "2012-10-30")) AS T
    ON T.date = W.date AND T.HR = W.HR
"""

In [121]:
with connection:
    q1 = connection.execute(QUERY_1) # busy hour is 6PM 18 with 310 orders

with connection:
    q2 = connection.execute(QUERY_2) # thursday is the busiest day for uber with 310 orders

with connection:
    q3 = connection.execute(QUERY_3) # thursday is the busiest day for uber with 310 orders

with connection:
    q4 = connection.execute(QUERY_4) # thursday is the busiest day for uber with 310 orders
    
with connection:
    q5 = connection.execute(QUERY_5) # thursday is the busiest day for uber with 310 orders
    
with connection:
    q6 = connection.execute(QUERY_6) # thursday is the busiest day for uber with 310 orders
    
for i in q6:
    print(i)



('2012-10-30', '00', 0.03, 13.0, None, None, None)
('2012-10-29', '15', 0.07, 26.0, None, None, None)
('2012-10-28', '03', 0.0, 9.0, None, None, None)
('2012-10-26', '16', 0.0, 0.0, None, None, None)
('2012-10-29', '04', 0.0, 15.0, None, None, None)
('2012-10-25', '19', 0.0, 0.0, None, None, None)
('2012-10-29', '12', 0.06, 20.0, None, None, None)
('2012-10-23', '19', 0.0, 0.0, None, None, None)
('2012-10-30', '12', 0.0, 7.0, None, None, None)
('2012-10-26', '22', 0.0, 0.0, None, None, None)
('2012-10-27', '20', 0.0, 7.0, None, None, None)
('2012-10-28', '21', 0.0, 14.0, None, None, None)
('2012-10-29', '13', 0.04, 28.0, None, None, None)
('2012-10-29', '01', 0.0, 11.0, None, None, None)


In [51]:
write_query_to_file(QUERY_1, "Query/Q1.sql")
write_query_to_file(QUERY_2, "Query/Q2.sql")
write_query_to_file(QUERY_3, "Query/Q3.sql")
write_query_to_file(QUERY_4, "Query/Q4.sql")
write_query_to_file(QUERY_5, "Query/Q5.sql")
write_query_to_file(QUERY_6, "Query/Q6.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)